In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import regex as re


In [2]:
def file_to_sentence_list(file_path):
	with open(file_path, 'r') as file:
		text = file.read()

	# Splitting the text into sentences using
	# delimiters like '.', '?', and '!'
	sentences = [sentence.strip() for sentence in re.split(
		r'(?<=[.!?])\s+', text) if sentence.strip()]

	return sentences

file_path = 'pizza.txt'
text_data = file_to_sentence_list(file_path)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1

# Create input sequences
input_sequences = []
for line in text_data:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

# Pad sequences and split into predictors and label
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(
	input_sequences, maxlen=max_sequence_len, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]

# Convert target data to one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=total_words)


In [4]:
# Define the model
model = Sequential()
model.add(Embedding(total_words, 10))
model.add(LSTM(128))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy',
			optimizer='adam', metrics=['accuracy'])


In [6]:
# Train the model
model.fit(X, y, epochs=50, verbose=1)


Epoch 1/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 71ms/step - accuracy: 0.0527 - loss: 5.6815
Epoch 2/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 52ms/step - accuracy: 0.0576 - loss: 5.4966
Epoch 3/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - accuracy: 0.0577 - loss: 5.4935
Epoch 4/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - accuracy: 0.0787 - loss: 5.4183
Epoch 5/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 7s 98ms/step - accuracy: 0.0661 - loss: 5.4187
Epoch 6/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - accuracy: 0.0725 - loss: 5.3023
Epoch 7/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 6s 67ms/step - accuracy: 0.0801 - loss: 5.1819
Epoch 8/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 5s 58ms/step - accuracy: 0.0950 - loss: 5.1027
Epoch 9/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - accuracy: 0.0833 - loss: 5.0072
Epoch 10/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - accuracy: 0.0883 - loss: 4.9644
Epoch 11/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 6s 68ms/step - accuracy: 0.0953 - loss: 4.9211
Epoch 12/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - accuracy:

In [7]:
# Generate next word predictions
seed_text = "Pizza have different "
next_words = 5

for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences(
		[token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted_probs = model.predict(token_list)
	predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
	seed_text += " " + predicted_word

print("Next predicted words:", seed_text)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Next predicted words: Pizza have different  become a symbol of comfort
